# Modelo de Recomendação: Predição de Próxima Rota

Este notebook implementa um sistema de recomendação para prever a próxima rota que um usuário
provavelmente escolherá, baseado em seu histórico de viagens e características comportamentais.
Utiliza XGBoost para classificação multiclasse com métricas Top-K accuracy.

In [18]:
!pip install xgboost scikit-learn pandas numpy --quiet


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [19]:

# Importações necessárias
import pandas as pd
import numpy as np
import pickle
import os
import json
from datetime import datetime

# Machine Learning
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, top_k_accuracy_score

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


In [20]:
# Carregamento e análise exploratória dos dados
df = pd.read_csv("dist/clusterization/dataset_com_clusters.csv")

print("Dados carregados com sucesso!")
print(f"Shape do dataset: {df.shape}")

# Verificar colunas necessárias
required_columns = ['fk_contact', 'date_purchase', 'route_departure']
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"Colunas obrigatórias ausentes: {missing_columns}")
else:
    print("Todas as colunas obrigatórias presentes")

print("\n" + "="*50)
print("PRIMEIRAS 5 LINHAS:")
df.head()


Dados carregados com sucesso!
Shape do dataset: (1741344, 29)
Todas as colunas obrigatórias presentes

PRIMEIRAS 5 LINHAS:


,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,...,route_return,is_round_trip,departure_company_freq,return_company_freq,origin_dept_freq,dest_dept_freq,route_departure_freq,cluster,data_clusterizacao,versao_modelo
0,bc02d5245bec63b30ff1102fa273fc03f58bc9cc3f674e...,a7218ff4ee7d37d48d2b4391b955627cb089870b934912...,2018-12-26,1900-01-01 15:33:35,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,50e9a8665b62c8d68bccc77c7c92431a1aa26ccbd38ed4...,0,0,8527a891e224136950ff32ca212b45bc93f69fbb801c3b...,1,...,0_to_0,1,20795,1548675,257893,209,140,2,2025-09-07 00:23:49.732200,KMeans_v1.0
1,5432f12612dd5d749b3be880e779989cf63b5efa4bcc4e...,37228485e0dc83d84d1bcd1bef3dc632301bf6cb22c8b5...,2018-12-05,1900-01-01 15:07:57,10e4e7caf8b078429bb1c80b1a10118ac6f963eff098fd...,e6d41d208672a4e50b86d959f4a6254975e6fb9b088116...,0,0,36ebe205bcdfc499a25e6923f4450fa8d48196ceb4fa0c...,1,...,0_to_0,1,2139,1548675,862,5,1,0,2025-09-07 00:23:49.732200,KMeans_v1.0
2,fb3caed9b2f1b6016d45ccddb19095476e61a2c85faa8e...,3467ec081e2421e72c96e7203b929d21927fd00b6b5f28...,2018-12-21,1900-01-01 18:41:54,7688b6ef52555962d008fff894223582c484517cea7da4...,8c1f1046219ddd216a023f792356ddf127fce372a72ec9...,0,0,ec2e990b934dde55cb87300629cedfc21b15cd28bbcf77...,1,...,0_to_0,1,69493,1548675,161597,3263,932,2,2025-09-07 00:23:49.732200,KMeans_v1.0
3,4dc44a6dd592b702feccb493d192210c86965aee684529...,ab3251a2be0f69713b8f97b0e9d1579e31551f4fd4facf...,2018-12-06,1900-01-01 14:01:38,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,d6acb3c1a79e57bcc03d976cb4d98f56edccd4cf426392...,0,0,5f9c4ab08cac7457e9111a30e4664920607ea2c115a143...,1,...,0_to_0,1,44132,1548675,61786,154,58,0,2025-09-07 00:23:49.732200,KMeans_v1.0
4,aa34ed7fd0a6b405df2df1bf9f8d68e6df9b9a868a6181...,ceea0de820a6379f2c4215bddaec66c33994b304607e56...,2021-02-23,1900-01-01 20:08:25,7688b6ef52555962d008fff894223582c484517cea7da4...,23765fc69c4e3c0b10f5d15471dc2245e2a19af16b513f...,0,0,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c...,1,...,0_to_0,1,12145,1548675,161597,21710,4018,0,2025-09-07 00:23:49.732200,KMeans_v1.0


In [21]:
# Preparação dos dados para modelo de recomendação
# Ordenar por usuário e data para criar sequências temporais
df = df.sort_values(by=["fk_contact", "date_purchase"])

# Criar variável target: próxima rota que o usuário escolherá
df["next_route_departure"] = df.groupby("fk_contact")["route_departure"].shift(-1)

# Remover registros sem próxima rota (última compra do usuário)
df = df.dropna(subset=["next_route_departure"])

print(f"Dataset após processamento: {df.shape}")
print(f"Usuários únicos: {df['fk_contact'].nunique():,}")
print(f"Rotas únicas: {df['route_departure'].nunique():,}")
print(f"Próximas rotas únicas: {df['next_route_departure'].nunique():,}")

print("\n" + "="*50)
print("AMOSTRA DOS DADOS PROCESSADOS:")
df[["fk_contact", "route_departure", "next_route_departure"]].head()


Dataset após processamento: (1159527, 30)
Usuários únicos: 270,778
Rotas únicas: 27,299
Próximas rotas únicas: 27,808

AMOSTRA DOS DADOS PROCESSADOS:


,fk_contact,route_departure,next_route_departure
1439179,000010ae2e13049769982d9f07de792d92452ff1d124e3...,f369cb89fc627e668987007d121ed1eacdc01db9e28f8b...,62f77e7d6197863ac98d9e0cfa76bea0c8e05379ed5281...
1465441,00007a5d618cd250d7f05766cfe01a8663a3767f1cd669...,7688b6ef52555962d008fff894223582c484517cea7da4...,81b8a03f97e8787c53fe1a86bda042b6f0de9b0ec9c093...
745484,00007a5d618cd250d7f05766cfe01a8663a3767f1cd669...,81b8a03f97e8787c53fe1a86bda042b6f0de9b0ec9c093...,81b8a03f97e8787c53fe1a86bda042b6f0de9b0ec9c093...
805592,00007a5d618cd250d7f05766cfe01a8663a3767f1cd669...,81b8a03f97e8787c53fe1a86bda042b6f0de9b0ec9c093...,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...
1583864,00008c39885815e42a0bb750cee199cd4da741a5645705...,7688b6ef52555962d008fff894223582c484517cea7da4...,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...


In [22]:
# Pré-processamento e codificação de variáveis
# Usar amostra para otimizar performance computacional
sample_size = min(50000, len(df))
df_model = df.sample(n=sample_size, random_state=42).copy()

# Identificar e processar colunas categóricas
categorical_cols = df_model.select_dtypes(include="object").columns.tolist()
if "nk_ota_localizer_id" in categorical_cols:
    categorical_cols.remove("nk_ota_localizer_id")

# Converter colunas que são numéricas mas foram lidas como object
numeric_but_object = ['fk_contact', 'place_origin_return', 'place_destination_return', 'fk_return_ota_bus_company']
for col in numeric_but_object:
    if col in categorical_cols:
        categorical_cols.remove(col)
        df_model[col] = pd.to_numeric(df_model[col], errors='coerce').fillna(0).astype(int)

# Codificar variáveis categóricas usando LabelEncoder
label_encoders = {}
for col in categorical_cols:
    if col in df_model.columns:
        le = LabelEncoder()
        df_model[col] = df_model[col].astype(str).fillna('unknown')
        df_model[col] = le.fit_transform(df_model[col])
        label_encoders[col] = le

print(f"Amostra processada: {len(df_model):,} registros")
print(f"Colunas categóricas codificadas: {len(categorical_cols)}")

print("\n" + "="*50)
print("AMOSTRA APÓS CODIFICAÇÃO:")
df_model.head()


Amostra processada: 50,000 registros
Colunas categóricas codificadas: 11

AMOSTRA APÓS CODIFICAÇÃO:


,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,...,is_round_trip,departure_company_freq,return_company_freq,origin_dept_freq,dest_dept_freq,route_departure_freq,cluster,data_clusterizacao,versao_modelo,next_route_departure
1146708,779c264f61b46cdc65b3db85dc2e63fe68f939ba15c284...,0,2524,32946,384,70,0,0,140,1,...,1,21117,1548675,4148,8037,565,2,0,0,5808
729276,49700983682c4febac3e65422fc788d6bec1fc4a70bfe2...,0,1228,21052,590,1412,0,0,206,1,...,1,27443,1548675,151,23673,76,1,0,0,4053
1647397,114570041930a0c1387cddee0f0c4fae927070fbe8c241...,0,3405,31874,987,10,0,0,59,1,...,1,190291,1548675,1711,2887,62,2,0,0,30
1717309,765f9ad006026b078e6328af5e8151e52fd96ddd80ebe4...,0,3464,31765,549,1412,0,0,185,1,...,1,6144,1548675,24124,23673,8585,0,0,0,3216
1677519,30f814d260c79f7742ebc9d01ed3c5f94fd44acfecac8b...,0,2400,14699,593,510,0,0,90,1,...,1,35355,1548675,257893,995,795,2,0,0,3853


In [23]:
# Filtro de rotas válidas - manter apenas rotas com pelo menos 2 ocorrências
route_counts = df_model["next_route_departure"].value_counts()
valid_routes = route_counts[route_counts >= 2].index
df_model = df_model[df_model["next_route_departure"].isin(valid_routes)]

print(f"Rotas válidas (≥2 ocorrências): {len(valid_routes):,}")
print(f"Dataset final: {len(df_model):,} registros")


Rotas válidas (≥2 ocorrências): 4,217
Dataset final: 45,769 registros


In [24]:
# Preparação das features e variável target
columns_to_drop = ["next_route_departure", "nk_ota_localizer_id"]
X = df_model.drop(columns=columns_to_drop)
y = df_model["next_route_departure"]

# Divisão treino/teste com estratificação
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Codificação da variável target
y_encoder = LabelEncoder()
y_train_enc = y_encoder.fit_transform(y_train)
y_test_enc = y_encoder.transform(y_test)

print(f"Features: {X.shape[1]} variáveis")
print(f"Conjunto de treino: {X_train.shape[0]:,} amostras")
print(f"Conjunto de teste: {X_test.shape[0]:,} amostras")
print(f"Classes únicas: {len(y_encoder.classes_):,}")
print(f"Amostras por classe (min/max): {np.bincount(y_train_enc).min()}/{np.bincount(y_train_enc).max()}")

print("\n" + "="*50)
print("DISTRIBUIÇÃO DAS ROTAS MAIS FREQUENTES:")
y.value_counts().head(10)



Features: 28 variáveis
Conjunto de treino: 36,615 amostras
Conjunto de teste: 9,154 amostras
Classes únicas: 4,217
Amostras por classe (min/max): 2/896

DISTRIBUIÇÃO DAS ROTAS MAIS FREQUENTES:


next_route_departure
4535    1120
3915    1110
3800     662
1334     625
3851     373
2577     340
4560     311
5163     298
3588     295
3776     275
Name: count, dtype: int64

In [25]:
# Treinamento do modelo XGBoost para classificação multiclasse
model = XGBClassifier(
    objective="multi:softprob",
    n_estimators=50,
    max_depth=3,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=1,
    verbosity=0
)

print("Treinando modelo XGBoost...")
model.fit(X_train, y_train_enc)
print("Treinamento concluído!")


Treinando modelo XGBoost...
Treinamento concluído!


In [26]:
# Avaliação do modelo com métricas Top-K
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)

# Calcular métricas de acurácia para sistemas de recomendação
all_classes = np.arange(y_proba.shape[1])
accuracy_top1 = accuracy_score(y_test_enc, y_pred)
accuracy_top5 = top_k_accuracy_score(y_test_enc, y_proba, k=5, labels=all_classes)
accuracy_top10 = top_k_accuracy_score(y_test_enc, y_proba, k=10, labels=all_classes)

print("="*60)
print("RESULTADOS DA AVALIAÇÃO DO MODELO")
print("="*60)
print(f"Top-1 Accuracy:  {accuracy_top1:.4f} ({accuracy_top1*100:.2f}%)")
print(f"Top-5 Accuracy:  {accuracy_top5:.4f} ({accuracy_top5*100:.2f}%)")
print(f"Top-10 Accuracy: {accuracy_top10:.4f} ({accuracy_top10*100:.2f}%)")
print("="*60)

print(f"\nESTATÍSTICAS DO TESTE:")
print(f"   • Amostras de teste: {len(y_test_enc):,}")
print(f"   • Classes preditas únicas: {len(np.unique(y_pred)):,}")
print(f"   • Classes reais únicas: {len(np.unique(y_test_enc)):,}")
print(f"   • Total de classes do modelo: {y_proba.shape[1]:,}")

RESULTADOS DA AVALIAÇÃO DO MODELO
Top-1 Accuracy:  0.2781 (27.81%)
Top-5 Accuracy:  0.5175 (51.75%)
Top-10 Accuracy: 0.5588 (55.88%)

ESTATÍSTICAS DO TESTE:
   • Amostras de teste: 9,154
   • Classes preditas únicas: 1,215
   • Classes reais únicas: 2,880
   • Total de classes do modelo: 4,217


In [27]:
# Salvamento dos artefatos do modelo para uso em API
os.makedirs("dist/recommendation/artifacts", exist_ok=True)

# Salvar modelo treinado
with open("dist/recommendation/artifacts/modelo_recomendacao.pkl", "wb") as f:
    pickle.dump(model, f)

# Salvar encoders
with open("dist/recommendation/artifacts/label_encoder.pkl", "wb") as f:
    pickle.dump(y_encoder, f)

with open("dist/recommendation/artifacts/feature_encoders.pkl", "wb") as f:
    pickle.dump(label_encoders, f)

print("Artefatos do modelo salvos com sucesso!")


Artefatos do modelo salvos com sucesso!


In [28]:
# Análise de importância das features
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

# Salvar importância das features
feature_importance.to_csv("dist/recommendation/artifacts/feature_importance.csv", index=False)

print("TOP 10 FEATURES MAIS IMPORTANTES:")
print("="*50)
for i, (_, row) in enumerate(feature_importance.head(10).iterrows(), 1):
    print(f"{i:2d}. {row['feature']:<25} {row['importance']:.4f}")

print(f"\nTotal de features analisadas: {len(feature_importance)}")


TOP 10 FEATURES MAIS IMPORTANTES:
 1. route_departure_freq      0.1631
 2. fk_departure_ota_bus_company 0.0727
 3. place_destination_departure 0.0699
 4. departure_company_freq    0.0630
 5. place_origin_departure    0.0553
 6. route_departure           0.0550
 7. dest_dept_freq            0.0500
 8. origin_dept_freq          0.0498
 9. return_company_freq       0.0393
10. gmv_success               0.0389

Total de features analisadas: 28


In [31]:
# Inferência em TODO o dataset (sem amostra) e export em lotes
print("Gerando dataset COMPLETO com predições (em lotes)...")

# Recarregar o dataset completo de clusterização
_df_full_raw = pd.read_csv("dist/clusterization/dataset_com_clusters.csv")

# Manter colunas de identificação mínimas no output
_id_cols = [c for c in [
    "nk_ota_localizer_id", "fk_contact", "date_purchase", "route_departure"
] if c in _df_full_raw.columns]
_output_id = _df_full_raw[_id_cols].copy()

# Usar exatamente as mesmas features do treino
_features = X_train.columns.tolist()
X_full = _df_full_raw[_features].copy()

# Corrigir tipos (mesma lógica do pré-processamento)
_numeric_but_object = [
    'fk_contact', 'place_origin_return', 'place_destination_return', 'fk_return_ota_bus_company'
]
for _col in _numeric_but_object:
    if _col in X_full.columns:
        X_full[_col] = pd.to_numeric(X_full[_col], errors='coerce').fillna(0).astype(int)

# Transformação segura com fallback para 'unknown'
_def_unknown = 'unknown'

def _transform_with_unknown(series: pd.Series, encoder: LabelEncoder) -> pd.Series:
    _values = series.astype(str).fillna(_def_unknown)
    _classes = list(encoder.classes_)
    _class_to_idx = {cls: idx for idx, cls in enumerate(_classes)}
    _unknown_idx = _class_to_idx.get(_def_unknown, None)
    _mapped = _values.map(lambda x: _class_to_idx.get(x, _unknown_idx if _unknown_idx is not None else -1))
    return _mapped.astype(int)

# Aplicar encoders de features quando disponível
for _col, _enc in label_encoders.items():
    if _col in X_full.columns:
        X_full[_col] = _transform_with_unknown(X_full[_col], _enc)

# Predição em lotes para uso eficiente de memória
_batch_size = 100000
_num_rows = len(X_full)
_topk = 5

_pred_cols = [f"predicted_route_{i}" for i in range(1, _topk + 1)]
_proba_cols = [f"prob_route_{i}" for i in range(1, _topk + 1)]

_csv_path = "dist/recommendation/dataset_recomendacoes_completo.csv"
_header_written = False

# Decodificadores para voltar às rotas originais
_target_feature_encoder = label_encoders.get("next_route_departure", None)
if _target_feature_encoder is None:
    raise RuntimeError("Encoder de 'next_route_departure' não encontrado para decodificar as predições.")

for _start in range(0, _num_rows, _batch_size):
    _end = min(_start + _batch_size, _num_rows)
    _Xb = X_full.iloc[_start:_end]

    _proba = model.predict_proba(_Xb)
    _top_idx = np.argsort(_proba, axis=1)[:, -_topk:][:, ::-1]

    # Mapear: índices de classe -> valores originais do y (inteiros) -> rotas (strings)
    _flat = _top_idx.ravel()
    _encoded_flat = y_encoder.inverse_transform(_flat)
    _encoded_matrix = _encoded_flat.reshape(_top_idx.shape)

    _routes = np.empty_like(_encoded_matrix, dtype=object)
    for i in range(_topk):
        _routes[:, i] = _target_feature_encoder.inverse_transform(_encoded_matrix[:, i])

    _out = _output_id.iloc[_start:_end].copy()
    for i in range(_topk):
        _out[_pred_cols[i]] = _routes[:, i]
        _out[_proba_cols[i]] = _proba[np.arange(len(_proba)), _top_idx[:, i]]

    _out.to_csv(_csv_path, index=False, mode=('w' if not _header_written else 'a'), header=(not _header_written))
    _header_written = True

print(f"Dataset COMPLETO salvo: {_num_rows:,} registros com predições")


Gerando dataset COMPLETO com predições (em lotes)...
Dataset COMPLETO salvo: 1,741,344 registros com predições


In [30]:
# Geração dos metadados do modelo
metadata = {
    "projeto": "Modelo de Recomendação - Predição de Próxima Rota",
    "versao_modelo": "XGBoost_v1.0",
    "data_criacao": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "algoritmo": "XGBoost Classifier",
    "dataset": {
        "total_amostra": len(df_model),
        "total_features": len(X_train.columns),
        "total_rotas_unicas": len(y_encoder.classes_),
        "features_utilizadas": X_train.columns.tolist()
    },
    "performance": {
        "accuracy_top1": float(accuracy_top1),
        "accuracy_top5": float(accuracy_top5),
        "accuracy_top10": float(accuracy_top10)
    },
    "distribuicao_rotas": {
        "total_rotas_validas": len(valid_routes),
        "rotas_mais_frequentes": int(route_counts.max()),
        "rotas_menos_frequentes": int(route_counts.min())
    },
    "arquivos_gerados": {
        "modelo_completo": "artifacts/modelo_recomendacao.pkl",
        "label_encoder": "artifacts/label_encoder.pkl",
        "feature_encoders": "artifacts/feature_encoders.pkl",
        "feature_importance": "artifacts/feature_importance.csv",
        "dataset_csv": "dataset_recomendacoes_completo.csv",
        "metadados": "metadata.json"
    }
}

with open("dist/recommendation/metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)


print("SISTEMA DE RECOMENDAÇÃO - TREINAMENTO CONCLUÍDO")

SISTEMA DE RECOMENDAÇÃO - TREINAMENTO CONCLUÍDO
